# User interface

In [1]:
from nilearn.plotting import plot_anat, show
import ipywidgets as widgets
import utils
import os
from importlib import reload
from nilearn import plotting
from nilearn import image as nli
reload(utils)
import nibabel as nib
import numpy as np
from ipywidgets import HBox, VBox
import numpy as np
from nilearn.plotting import plot_anat, show
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import ipywidgets as widgets
from IPython.display import display, Markdown

## Functions

### Misc functions

In [2]:


def display_markdown():
    # This function displays the markdown text, project_dict is 
    # a dictionary containing the project information
    markdown_text = f"""
    ### Experiment Data
    **User:** {project_dict['User']}  \n
    **Dataset:** {project_dict['Dataset']}  \n
    **Session:** {project_dict['Session']}  \n
    **Task:** {project_dict['Task']}  \n
    **Participants:** {project_dict['Participants']}  \n
    **Runs:** {project_dict['Runs']}  \n
    **Specie:** {project_dict['Specie']}  
    """

def create_input_widgets():
    input_widgets = {
        "User": widgets.Text(value='', description='User:'),
        "Dataset": widgets.Text(value='', description='Dataset:'),
        "Session": widgets.Text(value='', description='Session (optional):'),
        "Task": widgets.Text(value='', description='Task:'),
        "Participants": widgets.Text(value='', description='Participant numbers:'),
        "Runs": widgets.Text(value='', description='Runs available (1,2,3,...)'),
        "Specie": widgets.Text(value='', description='Specie (D or H):'),
    }
    return input_widgets

def update_data(b):
    global project_dict
    #clear_output(wait=True)
    project_dict = {key: widget.value for key, widget in input_widgets.items()}
    # update the project_dict in the screen
    update_project_dict()

def update_project_dict():
    row_2.clear_output()
    for key, value in project_dict.items():
        #print(f"{key}: {value}")
        with row_2:
            print(f"{key}: {value}")


def cancel_update(b):
    #clear_output(wait=True)
    for key, widget in input_widgets.items():
        widget.value = project_dict[key]
    
    # update the project_dict in the screen
    update_project_dict()



### Preprocessing functions

In [ ]:

def preprocess_run(sub_N, run_N, dataset, task, specie, datafolder, session, smooth=0, combination=['-x','z','-y']):
    """
    Preprocesses a single run of a single subject.
    Reorients file
    """

    # path to design.fsf, current directory followed by design.fsf
    design_path = os.path.join(os.getcwd(), 'FSL_designs' + os.sep + 'preprocess.fsf')

    ## determine input file and output directory ## 

    # input directory in BIDS format
    filename = datafolder + os.sep + dataset + os.sep + 'BIDS' + os.sep + specie + '-sub-' + str(sub_N).zfill(3) + os.sep + 'func' + os.sep
    # check if session is not empty
    if session != '':
        filename += 'ses-' + session + os.sep
    filename += specie + '-sub-' + str(sub_N).zfill(3)
    if session != '':
        filename += '_ses-' + session
    filename += '_task-' + task + '_run-' + str(run_N).zfill(2) + '_bold.nii.gz'

    # get TR and number of volumes
    TR,volumes = utils.extract_params(filename)

    # create output directory, where the fsl output will be saved (preprocessed data)
    outputdir = datafolder + os.sep + dataset + os.sep + 'preprocessing' + os.sep + specie + '-sub-' + str(sub_N).zfill(3) + os.sep + specie + '-sub-' + str(sub_N).zfill(3)
    # check if session is not empty
    if session != '':
        outputdir += '_ses-' + session
    outputdir += '_task-' + task + '_run-' + str(run_N).zfill(2)

    ## Filling out the design.fsf file ##
    # create list of labels to fill in the design.fsf file
    label_list = ['Outputdir', 'TR', 'Volumes', 'BET', 'Smooth', 'Input']

    # create dictionary to fill in the design.fsf file
    to_fill_dict = dict()
    for label in label_list:
        to_fill_dict[label] = dict()
        if label == 'Outputdir':
            to_fill_dict[label]['string_to_find'] = 'set fmri(outputdir)'
            to_fill_dict[label]['string_to_replace'] = ('set fmri(outputdir) "' + outputdir + '"')
        elif label == 'TR':
            to_fill_dict[label]['string_to_find'] = 'set fmri(tr)'
            to_fill_dict[label]['string_to_replace'] = ('set fmri(tr) ' + str(TR))
        elif label == 'Volumes':
            to_fill_dict[label]['string_to_find'] = 'set fmri(npts)'
            to_fill_dict[label]['string_to_replace'] = ('set fmri(npts) ' + str(volumes))
        elif label == 'BET':
            to_fill_dict[label]['string_to_find'] = 'set fmri(bet_yn)'
            if specie == 'H':
                to_fill_dict[label]['string_to_replace'] = ('set fmri(bet_yn) 1')
            elif specie == 'D':
                to_fill_dict[label]['string_to_replace'] = ('set fmri(bet_yn) 0')
        elif label == 'Smooth':
            to_fill_dict[label]['string_to_find'] = 'set fmri(smooth)'
            to_fill_dict[label]['string_to_replace'] = ('set fmri(smooth) ' + str(smooth))
        elif label == 'Input':
            to_fill_dict[label]['string_to_find'] = 'set feat_files(1)'
            to_fill_dict[label]['string_to_replace'] = ('set feat_files(1) "' + filename + '"')

    # fill in the design.fsf file
    design_path = os.path.join(os.getcwd(), 'FSL_designs'  + os.sep + 'preprocess.fsf')
    design_modified_path = os.path.join(os.getcwd(), 'FSL_designs'  + os.sep + 'preprocess_modified.fsf')

    utils.fill_fsf(to_fill_dict, design_path, design_modified_path)

    # run feat
    command = './feat ' + design_modified_path

    # check if system is windows, if so, do not execute command
    if os.name == 'nt': # Windows
        print("system is windows, command not executed. Command is:")
        print(command)
    else:
        os.system(command)
    
    ## reorient run ##

    base_filename = specie + '-sub-' + str(sub_N).zfill(3)
    # non-oriented file
    non_oriented_file = base_filename + '_not-oriented.nii.gz'
    # oriented file
    reoriented_file = base_filename + '_reoriented.nii.gz'

    #copy preprocessed_file to non_oriented_file
    #shutil.copyfile(outputdir + os.sep + preprocessed_file, outputdir + os.sep + non_oriented_file)
    print(non_oriented_file + ' created')
    print('Working directory: ' + outputdir)

    # check if system is windows, if so, do not execute command
    if os.name == 'nt': # Windows
        print("system is windows, command not executed.")
        print("orientation to use: " + combination)
        print("non-oriented file: " + outputdir + os.sep + non_oriented_file)
        print("oriented file: " + outputdir + os.sep + reoriented_file)
        
    utils.reorient_file(outputdir + os.sep + non_oriented_file, outputdir + os.sep + reoriented_file, combination)



In [3]:
# Define the dictionary and the display functions
project_dict = {
    "User": "",
    "Dataset": "",
    "Session": "",
    "Task": "",
    "Participants": "",
    "Runs": "",
    "Specie": "",
}

# Create the input widgets
input_widgets = create_input_widgets()
update_button = widgets.Button(description="Update")
cancel_button = widgets.Button(description="Cancel")

# Assign an event to the button
update_button.on_click(update_data)
cancel_button.on_click(cancel_update)

# Create a tab widget
tab = widgets.Tab()

# Create the rows to store the widgets
row_1 = widgets.VBox([widgets.VBox(list(input_widgets.values())), update_button, cancel_button])
row_2 = widgets.Output(layout={'border': '1px solid black'})                     

# Create the columns to store
col_1 = widgets.HBox([row_1,row_2])

# Organize contents into tabs
tab_contents = ['Data Entry', 'Tab 2', 'Tab 3']
tab_list = [col_1, 
            widgets.Text(description="Tab 2 Content"), 
            widgets.Text(description="Tab 3 Content")]
tab.children = tab_list

for i, name in enumerate(tab_contents):
    tab.set_title(i, name)

# Display the tab widget
display(tab)




In [4]:
import ipywidgets as widgets
from IPython.display import display

def create_button_matrix(rows, columns):
    # This function creates a button and assigns an on_click event
    def create_button(row, col):
        button = widgets.Button(description=f"Button {row+1},{col+1}")
        button.on_click(lambda b: print(f"Button at Row {row+1}, Column {col+1} was pressed"))
        return button

    # Create a grid of buttons
    grid = widgets.GridBox(children=[
        create_button(row, col) for row in range(rows) for col in range(columns)
    ], layout=widgets.Layout(grid_template_columns="repeat(4, 100px)"))
    
    display(grid)

# Create a 3x4 matrix of buttons
create_button_matrix(3, 4)


GridBox(children=(Button(description='Button 1,1', style=ButtonStyle()), Button(description='Button 1,2', styl…

Button at Row 1, Column 1 was pressed
Button at Row 1, Column 2 was pressed
Button at Row 1, Column 3 was pressed
Button at Row 1, Column 4 was pressed
Button at Row 2, Column 1 was pressed
Button at Row 2, Column 2 was pressed
Button at Row 2, Column 3 was pressed
Button at Row 2, Column 4 was pressed
Button at Row 3, Column 1 was pressed
Button at Row 3, Column 2 was pressed
Button at Row 3, Column 3 was pressed
Button at Row 3, Column 4 was pressed
Button at Row 2, Column 4 was pressed
Button at Row 1, Column 4 was pressed


In [6]:
# Create a tab widget
tab = widgets.Tab()

# Create contents for each tab
children = [widgets.Text(description=f"Tab {i+1}") for i in range(3)]

# Set the children for the tab widget
tab.children = children

# Name your tabs
for i in range(len(children)):
    tab.set_title(i, f'Tab {i+1}')

# Display the tab widget
display(tab)
